# Bering Sea with flexible biology: application setup

This script builds a working ROMS application environment with the following components:

 - Grid: Bering10K
 - Ocean model: nonlinear
 - Ice model: Budgell (Hedstrom version)
 - Biogeochemical models: 
   - BEST_NPZ
   - BIO_COBALT
   - BIO_BANAS

In [1]:
import os
import sys
import yaml
from collections import OrderedDict

sys.path.insert(0, '/Users/kakearney/Documents/PythonCodeKAK/romscom/romscom')

import varinfo

## Header file

The header file combines the defaults for the Bering_10K application along with all biological module options.  The bio-dependent flags are wrapped in a conditional such that the specific module to be compiled is turned on by the compilation script (build_roms.bash).  This allows the same header file to be used for all app compilation variants.  Ideally, the final header file should not be unless major changes are made to the Bering_10K ocean model configuration or one of the biogeochemical models.

In [2]:
# Header file name
# Note: must match application flag (BERING_10K)

newfile = "bering_10k.h"

# Files to add

berapps = "../.."
header_subfiles = [os.path.join(berapps, "subApps", "BERING_10K", "post_ROMS3p6", "bering_10k.h"), 
                   os.path.join(berapps, "subApps", "BIO_COBALT", "cobalt_cpp.h"), 
                   os.path.join(berapps, "subApps", "BEST_NPZ",   "bestnpz_cpp.h"),
                   os.path.join(berapps, "subApps", "BIO_BANAS",  "banas_cpp.h")]

# Concatenate sub-files into new header file

with open(newfile, 'w') as out:

    for file in header_subfiles:
        with open(file, 'r') as f:
            contents = f.read()
            out.write(contents)

## varinfo.dat

We build several different versions of the varinfo.dat file, to be used with different compiled versions.  The physics-only option combines I/O details for ocean and ice variables.  Each biological model adds biological I/O details to that physics-only option.  These are kept separate to ensure no variable conflicts occur between BGC options.

In [3]:
# Read default ocean I/O (note: variables organized into many topic arrays)

with open(os.path.join(berapps, "subApps", "ocean", "default_io.yaml"), 'r') as f:
    Vphys = yaml.load(f)

# Ice and BGC I/O files only hold a single metadata array, so read with varinfo.readfile wrapper to extract just that
    
Vice = varinfo.readfile(os.path.join(berapps, "subApps", "IceBudgell", "ice_io.yaml"),     type="yaml")

Vbes = varinfo.readfile(os.path.join(berapps, "subApps", "BEST_NPZ",   "bestnpz_io.yaml"), type="yaml")
Vcob = varinfo.readfile(os.path.join(berapps, "subApps", "BIO_COBALT", "cobalt_io.yaml"),  type="yaml")
Vban = varinfo.readfile(os.path.join(berapps, "subApps", "BIO_BANAS",  "banas_io.yaml"),   type="yaml")

# Combine physics variables (splitting based on where bio variables will be inserted)

V1 = Vphys['state_variables'] + \
     Vphys['wet_dry_masks'] + \
     Vphys['detided'] + \
     Vphys['momentum'] + \
     Vphys['tracer_flux'] + \
     Vphys['mixing'] + \
     Vphys['boundary'] + \
     Vphys['atmos_forcing'] + \
     Vphys['tidal_forcing'] + \
     Vphys['river_forcing'] + \
     Vphys['boundary_layer'] + \
     Vphys['climatology'] + \
     Vphys['adjoint'] + \
     Vphys['tangent'] + \
     Vphys['observation_vars'] + \
     Vphys['passive'] + \
     Vphys['sediment']
        
V2 = Vphys['quadratic'] + \
     Vphys['diagnostic'] + \
     Vphys['forward_trajectory'] + \
     Vphys['assimilation']

    
# Rename bottom drag coefficient (in sediment) to match Bering10K grid file (rdrag -> rdrag_grid)

for ii in V1:
    if ii['index_code'] == "idragL":
        ii['variable'] = "rdrg_grid"
        
# Combine ocean and ice I/O

Vbase = V1 + V2 + Vice

# Combine with bio options.  Note that bio state variables need to preceed any variable 
# that includes (itrc) in their ID (e.g. tracer2 and others in quadratic set). I opt to 
# place all tracers together just before quadratic terms, and other bio variables at the end.

trc = [];
oth = []
for ii in Vbes:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vbes = V1 + trc + V2 + Vice + oth


trc = [];
oth = []
for ii in Vcob:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vcob = V1 + trc + V2 + Vice + oth

trc = [];
oth = []
for ii in Vban:
    if ii['index_code'].startswith('idTvar(') | \
       ii['index_code'].startswith('idBeTvar(') | \
       ii['index_code'].startswith('idBeTvar('):
        trc.append(ii)
    else:
        oth.append(ii)
        
Vban = V1 + trc + V2 + Vice + oth

# Write to files (classic format)

varinfo.writefile(Vbase, "varinfo_phys.dat",    type="classic")
varinfo.writefile(Vbes,  "varinfo_bestnpz.dat", type="classic")
varinfo.writefile(Vcob,  "varinfo_cobalt.dat",  type="classic")
varinfo.writefile(Vban,  "varinfo_banas.dat",   type="classic")

## Input file templates

Input files (including variants on ocean.in, ipar.in, and bpar.in) were copied from the following locations and then modified manually as follows:

- bering_ocean.yaml: copy of subApps/BERING_10K_new/ocean_bering10k_new.in.yaml, with customized time variables, archiving flags, input .nc files, and output .nc filenames.
- bering_ipar.yaml: copy of subApps/IceBudgell/ice_budgell.in.yaml
- bering_bpar_bestnpz.yaml: copy of subApps/BEST_NPZ/bestnpz_bpar.yaml
- bering_bpar_cobalt.yaml: copy of subApps/BIO_COBALT/cobalt_bpar.yaml
- bering_bpar_banas.yaml: copy of subApps/BIO_BANAS/banas_bpar.yaml

